In [1]:
import os
import gc
import pickle
import networkx as nx
import igraph as ig
import pandas as pd
# from collections import defaultdict
os.chdir("/home/yaroslav/FCUL/MARS_1.0")

In [2]:
from project_cda.anime_graph_builder import AnimeGraphBuilder
from project_cda.community_tracker import CommunityTracker
from project_cda.cluster_evaluation import ClusterEvaluation
from project_cda.partition_enricher import PartitionEnricher
from project_cda.cluster_visualizer import ClusterVisualizer

In [3]:
DATA_DIR = "data"
USERS_CSV_PATH = f"{DATA_DIR}/datasets/anime_azathoth42/users_sterilized.csv"
USER_DICT_PATH = f"{DATA_DIR}/helpers/user_dict_filtered.json"   # 95 percentile
ANIME_CSV_PATH = f"{DATA_DIR}/datasets/anime_azathoth42/anime_sterilized.csv"

## EDGING SETTINGS (keep **ONE** option uncommented)

In [4]:
# === EDGES SETTINGS ===
def get_edges_config():
    # --- METHOD: Jaccard + KNN ---
    return {
        "method": "jaccard",
        "threshold": 0.05
    }

    # --- METHOD: Raw / Projected ---
    # return {
    #     "method": "raw",
    #     "threshold": 0,
    # }

## SPARSING SETTINGS (keep **ONE** option uncommented)

In [5]:

def get_sparsing_config():
    # --- METHOD: No sparsing ---
    return {}

    # --- METHOD: KNN ---
    # return {
    #     "type": "knn",
    #     "k": 10,
    # }

    # --- METHOD: Backbone ---
    # return {
    #     "type": "backbone",
    #     "alpha": 0.05
    # }

## CLUSTERING ALGORITHM SETTINGS (keep **ONE** option uncommented)

In [ ]:
# === НАСТРОЙКИ АЛГОРИТМА ===
def get_algo_config():
    # --- 1. LEIDEN: MODULARITY (Классика) ---
    # return {
    #     "name": "leiden_mod",
    #     "kwargs": {
    #         "objective_function": "modularity",
    #         "resolution": 1.0,  # "Gamma". 1.0 - стандарт. Больше - мельче кластеры.
    #         "n_iterations": -1            # -1 = крутить до сходимости (рекомендуется)
    #     }
    # }

    # --- 2. LEIDEN: CPM (Constant Potts Model) ---
    # return {
    #     "name": "leiden_cpm",
    #     "kwargs": {
    #         "objective_function": "CPM",
    #         "resolution_parameter": 0.05, # ВАЖНО: Это порог плотности. Значения: 0.01, 0.05, 0.1...
    #         "n_iterations": -1
    #     }
    # }

    # --- 3. LEADING EIGENVECTOR (Спектральный) ---
    # return {
    #     "name": "eigenvector",
    #     "kwargs": {
    #         # clusters=None -> алгоритм сам решит, сколько кластеров, опираясь на модулярность
    #         "clusters": None 
    #     }
    # }

    # --- 4. WALKTRAP (Random Walks) ---
    # return {
    #     "name": "walktrap",
    #     "kwargs": {
    #         "steps": 4  # Длина блуждания. Мало (3-4) -> мелкие кластеры. Много (8-10) -> крупные.
    #     }
    # }

    # --- 5. INFOMAP (Flow-based) ---
    # return {
    #     "name": "infomap",
    #     "kwargs": {
    #         "trials": 10  # Количество попыток. Больше -> стабильнее результат.
    #     }
    # }

    # --- 6. LABEL PROPAGATION ---
    return {
        "name": "label_propagation",
        "kwargs": {} # У него почти нет параметров, можно weights передать позже
    }

In [7]:
def make_experiment_name(edge_conf, sparse_conf, algo_conf):
    parts = []
    
    # 1. Edges part (e.g., "Jac005")
    e_name = edge_conf['method'][:3].capitalize()
    th_str = str(edge_conf['threshold']).replace('.', '')
    parts.append(f"{e_name}{th_str}")
    
    # 2. Spars part (e.g., "KNN20")
    s_type = sparse_conf.get('type')
    if s_type == 'knn':
        k = sparse_conf.get('k')
        parts.append(f"KNN{k}")
    elif s_type == 'backbone':
        a = str(sparse_conf.get('alpha')).replace('.', '')
        parts.append(f"BB{a}")
    else:
        parts.append("Full")
        
    # 3. Algo part (e.g., "LeidenM10")
    algo = algo_conf['name']
    if algo == 'leiden':
        res = str(algo_conf['kwargs'].get('resolution_parameter', 1.0)).replace('.', '')
        parts.append(f"LMod{res}")
    elif algo == 'infomap':
        parts.append(f"InfoT{algo_conf['kwargs'].get('trials', 1)}")
    else:
        parts.append(algo.capitalize())
        
    return "_".join(parts)

In [8]:
if not os.path.exists(DATA_DIR):
    os.makedirs(DATA_DIR)

GRAPH_DIR = f"{DATA_DIR}/graphs/"
if not os.path.exists(GRAPH_DIR):
    os.makedirs(GRAPH_DIR)

REPORT_DIR = f"{DATA_DIR}/reports/"
if not os.path.exists(REPORT_DIR):
    os.makedirs(REPORT_DIR)

PARTITION_DIR = f"{DATA_DIR}/partitions/"
if not os.path.exists(PARTITION_DIR):
    os.makedirs(PARTITION_DIR)

PLOTS_DIR = f"{DATA_DIR}/plots/"
if not os.path.exists(PLOTS_DIR):
    os.makedirs(PLOTS_DIR)

In [9]:
# 1. Забираем настройки
EDGES_CONF = get_edges_config()
SPARS_CONF = get_sparsing_config()
ALGO_CONF = get_algo_config()

# 2. Формируем имя
EXP_NAME = make_experiment_name(EDGES_CONF, SPARS_CONF, ALGO_CONF)
CURRENT_EXP_GRAPH_DIR = os.path.join(GRAPH_DIR, EXP_NAME)
CURRENT_EXP_REPORT_DIR = os.path.join(REPORT_DIR, EXP_NAME)
CURRENT_EXP_PARTITION_DIR = os.path.join(PARTITION_DIR, EXP_NAME)
CURRENT_EXP_PLOTS_DIR = os.path.join(PLOTS_DIR, EXP_NAME)

if not os.path.exists(CURRENT_EXP_GRAPH_DIR):
    os.makedirs(CURRENT_EXP_GRAPH_DIR)
if not os.path.exists(CURRENT_EXP_REPORT_DIR):
    os.makedirs(CURRENT_EXP_REPORT_DIR)
if not os.path.exists(CURRENT_EXP_PARTITION_DIR):
    os.makedirs(CURRENT_EXP_PARTITION_DIR)
if not os.path.exists(CURRENT_EXP_PLOTS_DIR):
    os.makedirs(CURRENT_EXP_PLOTS_DIR)

print(f"EDGES CONFIG:           {EDGES_CONF}")
print(f"SPARSING CONFIG:        {SPARS_CONF}")
print(f"SPARSING CONFIG:        {ALGO_CONF}")
print(f"EXPERIMENT:             {EXP_NAME}")
print(f"OUTPUT GRAPH PATH:      {CURRENT_EXP_GRAPH_DIR}")
print(f"OUTPUT PARTITIONS PATH: {CURRENT_EXP_PARTITION_DIR}")
print(f"OUTPUT REPORT PATH:     {CURRENT_EXP_REPORT_DIR}")
print(f"OUTPUT PLOT PATH:       {CURRENT_EXP_PLOTS_DIR}")

EDGES CONFIG:           {'method': 'jaccard', 'threshold': 0.05}
SPARSING CONFIG:        {}
SPARSING CONFIG:        {'name': 'leiden_mod', 'kwargs': {'objective_function': 'modularity', 'resolution': 1.0, 'n_iterations': -1}}
EXPERIMENT:             Jac005_Full_Leiden_mod
OUTPUT GRAPH PATH:      data/graphs/Jac005_Full_Leiden_mod
OUTPUT PARTITIONS PATH: data/partitions/Jac005_Full_Leiden_mod
OUTPUT REPORT PATH:     data/reports/Jac005_Full_Leiden_mod
OUTPUT PLOT PATH:       data/plots/Jac005_Full_Leiden_mod


In [10]:
graph_builder = AnimeGraphBuilder(users_csv_path=USERS_CSV_PATH,
                                 user_dict_json_path=USER_DICT_PATH,
                                 anime_csv_path=ANIME_CSV_PATH)

In [11]:
partitions_by_year = {}
for year in range(2006, 2008):
    print(f"\n>>> Processing {year}...")

    # --- 1. FILENAME GENERATION ---
    # Имя зависит ТОЛЬКО от года и настроек построения ребер (Edges Config)
    # Спарсинг и Алгоритмы не влияют на имя базового файла графа
    e_method = EDGES_CONF['method']
    e_thresh = str(EDGES_CONF['threshold']).replace('.', '')
    
    # Пример: base_2013_jaccard_005.gpickle
    base_graph_filename = f"base_{year}_{e_method}_{e_thresh}.gpickle"
    base_graph_path = os.path.join(CURRENT_EXP_GRAPH_DIR, base_graph_filename)
    
    G = None

    # --- 2. LOADING OR BUILDING TARGET GRAPH ---
    if os.path.exists(base_graph_path):
        # --- Load a graph... ---
        print(f"Loading cached graph from {base_graph_filename}...")
        with open(base_graph_path, "rb") as f: G = pickle.load(f)
    else:
        print(f"Building graph from scratch for {year}...")
        # --- Build a graph... ---
        # ...using EDGES_CONF
        edges, counts = graph_builder.build_edges(year=year, **EDGES_CONF)   # Автоматически подставит method="jaccard", threshold=0.05
        G = graph_builder.build_graph(edges, counts, output_path=base_graph_path)  # Сохраняем полный граф в кэш
        del edges, counts; gc.collect()

    # --- 3. SPARSING ---  
    # --- 3. ФИЛЬТРАЦИЯ (Sparsification) ---
    # Работаем уже с объектом G (загруженным или только что созданным)
    s_type = SPARS_CONF.get('type') # Безопасное получение, если dict пустой -> None

    if s_type:
        spars_filename = f"sparse_{year}_{s_type}.gpickle"
        spars_path = os.path.join(CURRENT_EXP_GRAPH_DIR, spars_filename)

        if os.path.exists(spars_path):
             print(f"Sparse graph loaded from cache: {spars_filename}")
             with open(spars_path, "rb") as f: G = pickle.load(f)
        else:
            # Вызываем нужный метод
            spars_args = {k: v for k, v in SPARS_CONF.items() if k != 'type'}
            spars_args['output_path'] = spars_path # 👇 Подкидываем путь для сохранения


            if s_type == "knn":
                print(f"Sparsifying: KNN (k={SPARS_CONF.get('k')})")
                # Передаем параметры, исключая 'type'
                G = graph_builder.sparsify_knn(G, **spars_args)
        
            elif s_type == "backbone":
                print(f"Sparsifying: Backbone (alpha={SPARS_CONF.get('alpha')})")
                G = graph_builder.sparsify_backbone(G, **spars_args)
        
    else:
        print("No sparsification applied. Using full graph.")

    # --- КЛАСТЕРИЗАЦИЯ ---
    print(f"Clustering with {ALGO_CONF['name']}...")

    # ✅ Правильная конвертация NX -> iGraph
    # Сохраняет атрибут 'weight' и имена узлов (в атрибуте '_nx_name')
    h = ig.Graph.from_networkx(G)
    
    # Проверяем веса
    weights = h.es['weight'] if 'weight' in h.edge_attributes() else None

    algo_name = ALGO_CONF['name']
    algo_args = ALGO_CONF['kwargs']
    
    partition = None
    
    try:
        if algo_name in ['leiden_mod', 'leiden_cpm']:
            # Leiden вызывается одинаково, параметры внутри kwargs разные
            partition = h.community_leiden(weights=weights, **algo_args)
            
        elif algo_name == 'eigenvector':
            partition = h.community_leading_eigenvector(weights=weights, **algo_args)
            
        elif algo_name == 'walktrap':
            wc = h.community_walktrap(weights=weights, **algo_args)
            partition = wc.as_clustering()
            
        elif algo_name == 'infomap':
            partition = h.community_infomap(edge_weights=weights, **algo_args)

        elif algo_name == 'label_propagation':
            partition = h.community_label_propagation(weights=weights) # kwargs пустые обычно

        else:
            raise ValueError(f"Unknown algo: {algo_name}")

        # Конвертируем VertexClustering в удобный словарь {anime_id: cluster_id}
        cluster_dict = {}
        for idx, cluster_id in enumerate(partition.membership):
            original_id = h.vs[idx]['_nx_name'] # Имя из NetworkX
            cluster_dict[original_id] = cluster_id

    except Exception as e:
        print(f"Error: {e}")


    # Возьми любой узел из графа (или из result_partition)
    node_id = list(G.nodes())[0]

    print(f"ID узла: {node_id}, Тип: {type(node_id)}")
    print("Атрибуты узла:", G.nodes[node_id])


    del G, h, partition
    gc.collect()
    partitions_by_year[year] = cluster_dict


>>> Processing 2006...
Loading cached graph from base_2006_jaccard_005.gpickle...
No sparsification applied. Using full graph.
Clustering with leiden_mod...
ID узла: 457, Тип: <class 'int'>
Атрибуты узла: {'title': '457', 'popularity': 5}

>>> Processing 2007...
Loading cached graph from base_2007_jaccard_005.gpickle...
No sparsification applied. Using full graph.
Clustering with leiden_mod...
ID узла: 1, Тип: <class 'int'>
Атрибуты узла: {'title': '1', 'popularity': 1027}


In [12]:
def nx_to_igraph(G_nx):
    
    mapping = {n: i for i, n in enumerate(G_nx.nodes())} 
    edges = [(mapping[u], mapping[v]) for u, v in G_nx.edges()]
    g = ig.Graph(edges, directed=False)  
    
    
    if nx.get_edge_attributes(G_nx, 'weight'):
        g.es['weight'] = [G_nx[u][v].get('weight', 1) for u, v in G_nx.edges()]
    else:
        g.es['weight'] = [1] * len(G_nx.edges())
    
   
    g.vs['name'] = list(G_nx.nodes())
    
    return g, mapping

In [13]:
base_partition_filename = f"partition_{EXP_NAME}.csv"
base_partition_path = os.path.join(CURRENT_EXP_PARTITION_DIR, base_partition_filename)

tracked_communities = CommunityTracker.track_communities(partition_by_year=partitions_by_year, threshold=0.2)
CommunityTracker.save_aligned_history_to_csv(tracked_communities, base_partition_path)

Aligning year 2006...
Aligning year 2007...
Saved partition detail to data/partitions/Jac005_Full_Leiden_mod/partition_Jac005_Full_Leiden_mod.csv
Saved partition stats to  data/partitions/Jac005_Full_Leiden_mod/partition_Jac005_Full_Leiden_mod_stats.csv


In [14]:
enricher = PartitionEnricher(metadata_path=ANIME_CSV_PATH, key_col="anime_id", set_cols=["genres", "studio"])
anime_meta_dict = enricher.get_metadata_dict()

partition_enriched = enricher.enrich_partition(f"{CURRENT_EXP_PARTITION_DIR}/{base_partition_filename}")

In [15]:
print(partition_enriched.head())

   year  anime_id  cluster_id  Unnamed: 0  \
0  2006       457           0         387   
1  2006       558           0         469   
2  2006       565           0         475   
3  2006       846           0         650   
4  2006       853           0         656   

                                           image_url  score  scored_by  \
0  https://myanimelist.cdn-dena.com/images/anime/...   8.74     147314   
1  https://myanimelist.cdn-dena.com/images/anime/...   8.36      26374   
2  https://myanimelist.cdn-dena.com/images/anime/...   7.42      28053   
3  https://myanimelist.cdn-dena.com/images/anime/...   8.11      65095   
4  https://myanimelist.cdn-dena.com/images/anime/...   8.34     335137   

     rank                               opening_theme  \
0    38.0       ['"The Sore Feet Song" by Ally Kerr']   
1   188.0  ['"Saraba Aoki Omakage" by Road of Major']   
2  1951.0                                          []   
3   426.0  ['"Sentimental Generation" by Ami Tokito']   

In [16]:
cluster_evaluator = ClusterEvaluation(EXP_NAME, tracked_communities, anime_info=anime_meta_dict)
cluster_evaluator.evaluate()
base_evaluation_filename = f"evaluation_{EXP_NAME}.csv"
base_evaluation_path = os.path.join(CURRENT_EXP_REPORT_DIR, base_evaluation_filename)
evaluation_df = pd.DataFrame(cluster_evaluator.evaluation)
evaluation_df.to_csv(base_evaluation_path, index=False, encoding='utf-8')
print(evaluation_df)

Evaluating method: Jac005_Full_Leiden_mod...
                   Method  Avg_Gini  Avg_Entropy  Stability_AMI  \
0  Jac005_Full_Leiden_mod    0.3983       1.8452         0.1188   

   Count_Volatility  Purity_Source  Purity_Genre  
0               0.0         0.4906        0.5443  


In [17]:
viz = ClusterVisualizer(partition_enriched)

# A. Sankey
viz.plot_sankey(
    filename=os.path.join(CURRENT_EXP_PLOTS_DIR, "sankey.html"),
    key_col="anime_id",
    name_col="title",
    feature_cols=["genres", "studio"],
    metric_col="score",
    sort_col="members",
    age_col="year_start"
    )

# B. Streamgraph
viz.plot_streamgraph(
    filename=os.path.join(CURRENT_EXP_PLOTS_DIR, "stream.html"),
    feature_col="genres", 
    title="Rise and Fall of Anime Genres"
)

# B. Радар (Сравнение Кластеров 0, 1, 2 в 2010 году)
viz.plot_radar(
    filename=os.path.join(CURRENT_EXP_PLOTS_DIR, "radar.html"),
    year=2010, 
    target_clusters=[0, 1, 2], 
    feature_col="genres"
)

# C. Bubbles (Optional)
viz.plot_bubbles(
    filename=os.path.join(CURRENT_EXP_PLOTS_DIR, "bubbles.html"),
    x_col="score",      # Чем правее, тем выше оценка
    y_col="members",    # Чем выше, тем популярнее
    size_col="count",   # Размер = кол-во тайтлов
    title="Anime Landscape: Quality vs Popularity"
)

# D. Солнце (Иерархия)
viz.plot_sunburst(
    filename=os.path.join(CURRENT_EXP_PLOTS_DIR, "sunburst.html"),
    feature_col="source", # Год -> Кластер -> Манга/Оригинал
    title="Anime Source Hierarchy"
)

Generating Sankey diagram (Evolution)...
Plot saved to data/plots/Jac005_Full_Leiden_mod/sankey.html
Generating Streamgraph (Rise and Fall of Anime Genres)...
Saved: data/plots/Jac005_Full_Leiden_mod/stream.html
Generating Radar Chart (Cluster DNA Comparison)...
Saved: data/plots/Jac005_Full_Leiden_mod/radar.html
Generating Bubble Chart (Anime Landscape: Quality vs Popularity)...
Saved: data/plots/Jac005_Full_Leiden_mod/bubbles.html
Generating Sunburst (Anime Source Hierarchy)...
Saved: data/plots/Jac005_Full_Leiden_mod/sunburst.html
